In [1]:
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
from keras_tqdm import TQDMNotebookCallback
from keras import regularizers
from keras.models import load_model, Model, Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Conv2D, Input, Lambda, add
from subpixel import *
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.optimizers import Adam, RMSprop
from utilities import random_crop, ssim, content_fn, test_edsr, ImageLoader
%matplotlib inline

Using TensorFlow backend.


In [2]:
dtype = 'float32'
K.set_floatx(dtype)

if dtype == 'float16':
    K.set_epsilon(1e-4)

def vgg_preprocess_input(x):
    # BGR Mean values
    vgg_mean = np.array([103.939, 116.779, 123.68], dtype=dtype).reshape((1,1,3))
    return x - vgg_mean


# get output of vgg conv layers function

path = './images/'


bs = 8
scale = 2 # 3 or 4
patch_size = 96
img_size = int(patch_size * scale)

IMAGES = sorted(glob.glob(path + '*'))
sr_trn_gen = ImageLoader(IMAGES, patch_size, scale, batchSize = bs)

In [3]:
scale = 4
input_shape = (48, 48, 3)
n_feats = 256
n_resblocks = 32

# def EDSR(scale = 4, input_shape = (48, 48, 3), n_feats = 256, n_resblocks = 32):
def res_block(input_tensor, nf, res_scale = 1.0):
    x = Conv2D(nf, (3, 3), padding='same', activation = 'relu', \
               activity_regularizer=regularizers.l1(10e-10))(input_tensor)
    x = Conv2D(nf, (3, 3), padding='same', activity_regularizer=regularizers.l1(10e-10))(x)
    x = Lambda(lambda x: x * res_scale)(x)
    x = add([x, input_tensor])
    return x
    
# model = Sequential()
# model.add(Conv2D(n_feats, kernel_size=3, input_shape=input_shape, padding='same', \
#                  activity_regularizer=regularizers.l1(10e-10)))
inp = Input(shape = input_shape)

x = Conv2D(n_feats, 3, padding='same', activity_regularizer=regularizers.l1(10e-10))(inp)
conv1 = x

if n_feats == 256:
    res_scale = 0.1
else:
    res_scale = 1.0

for i in range(n_resblocks):
    x = res_block(x, n_feats, res_scale)
    
x = Conv2D(n_feats, 3, padding='same', activity_regularizer=regularizers.l1(10e-10))(x)
x = add([x, conv1])

if not scale%2:
    for i in range(int(np.log2(scale))):
        x = Subpixel(n_feats, 3, 2, padding='same', activity_regularizer=regularizers.l1(10e-10))(x)
else:
    x = Subpixel(n_feats, 3, 3, padding='same', activity_regularizer=regularizers.l1(10e-10))(x)

sr = Conv2D(input_shape[-1], 1, padding='same', activity_regularizer=regularizers.l1(10e-10))(x)
model = Model(inputs=inp, output=sr, name='SR')


/home/seungho/anaconda3/envs/vdsr/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., name="SR", outputs=Tensor("co...)`


In [5]:
for layer in model.layers:
    if type(layer) == Subpixel:
        c, b = layer.get_weights()
        if scale == 3:
            w = icnr_weights(scale=3, shape=c.shape, dtype = dtype)
        else:
            w = icnr_weights(scale=2, shape=c.shape, dtype = dtype)
        layer.set_weights([w, b])

TypeError: __call__() takes from 2 to 3 positional arguments but 4 were given